In [43]:
import pandas as pd
import numpy as np
import time
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [44]:
operation_TRAIN = pd.read_csv("../data/operation_TRAIN.csv")
transaction_TRAIN = pd.read_csv("../data/transaction_TRAIN.csv")
tag_TRAIN = pd.read_csv("../data/tag_TRAIN.csv")                                                          
operation_TRAIN = operation_TRAIN.drop_duplicates("UID", keep="last")
transaction_TRAIN = transaction_TRAIN.drop_duplicates("UID", keep="last")
train_X = pd.merge(operation_TRAIN, transaction_TRAIN, on="UID", how="outer")
train = pd.merge(train_X, tag_TRAIN, on="UID", how="left")
train = train.sort_values("UID")

operation_round1 = pd.read_csv("../data/operation_round1.csv")
transaction_round1 = pd.read_csv("../data/transaction_round1.csv")                                                                                                                      
operation_round1 = operation_round1.drop_duplicates(["UID"], keep="last")
transaction_round1 = transaction_round1.drop_duplicates(["UID"], keep="last")                                                              
test_C = pd.merge(operation_round1, transaction_round1, on="UID", how="outer")
test_C = test_C.sort_values("UID")

In [45]:
del operation_TRAIN
del transaction_TRAIN
del tag_TRAIN
del operation_round1
del transaction_round1

In [46]:
test_C["Tag"] = -1
data = pd.concat([train, test_C])
data = data.fillna(value=-99999)
# data = data.drop(["time_x", "time_y"], axis=1)
feature = data.columns.drop(["UID", "Tag"])
col_feature = ["day_x", "day_y", "bal", "trans_amt"]
dis_feature = feature.drop(col_feature)

for col in dis_feature:
    lbl = preprocessing.LabelEncoder()
    data[col] = lbl.fit_transform(list(data[col].values))
    
train = data[data["Tag"] != -1]
train_Y = train["Tag"]
train_X = train.drop(["UID", "Tag"], axis=1)
test = data[data["Tag"] == -1]
test_UID = test["UID"]
test_X = test.drop(["UID", "Tag"], axis=1)

In [47]:
params = {'boosting_type': 'gbdt',
         'objective': 'binary',
         'metric': 'auc',
         'min_child_weight': 1.5,
         'num_leaves': 2 ** 5,
         'lambda_l2': 10,
         'subsample': 0.85,
         'learning_rate': 0.1,
         'seed': 2018,
         'colsample_bytree': 0.5,
         'nthread': 12}

x_train, x_val, y_train, y_true = train_test_split(train_X, train_Y, test_size=0.2, random_state=2018)

d_train = lgb.Dataset(x_train, y_train)

a_model = lgb.train(params, d_train, 50000)   
y_predict = a_model.predict(x_val)


In [48]:
d = pd.DataFrame()
d['prob'] = list(y_predict)
d['y'] = list(y_true)
d = d.sort_values(['prob'], ascending=[0])
y = d.y
PosAll = pd.Series(y).value_counts()[1]
NegAll = pd.Series(y).value_counts()[0]
pCumsum = d['y'].cumsum()
nCumsum = np.arange(len(y)) - pCumsum + 1
pCumsumPer = pCumsum / PosAll
nCumsumPer = nCumsum / NegAll
TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]

result = 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3
print("*******************************************************************************************************")
print("result:", result)
print("*******************************************************************************************************")

*******************************************************************************************************
result: 0.7099378881987577
*******************************************************************************************************


In [49]:
d_all = lgb.Dataset(train_X, label=train_Y)
model = lgb.train(params, d_all, 50000)

In [50]:
prob = model.predict(test_X)

In [51]:
result = pd.DataFrame({"UID": test_UID, "Tag": prob})

In [52]:
result.to_csv("../submission/" + str(int(time.time())) + ".csv", index=False)